In [1]:
import pandas as pd
import numpy as np
import random
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.environ import SolverFactory
import yfinance as yf
import matplotlib.pyplot as plt


In [58]:
model = pyo.ConcreteModel()

model.produtos = pyo.Set(initialize = ['Produto 1','Produto 2','Produto 3'])
model.maquinas = pyo.Set(initialize = ['Fresadora','Torno','Retificadora'])

model.x = pyo.Var(model.produtos, bounds=(0,None), domain=pyo.NonNegativeIntegers)

#Dicionarios

tempo_disponivel = {
    'Fresadora':500,
    'Torno':350,
    'Retificadora':150
}

horas_necessarias = {
    'Produto 1':{
        'Fresadora':9,
        'Torno':5,
        'Retificadora':3
    },
    'Produto 2':{
        'Fresadora':3,
        'Torno':4,
        'Retificadora':0
    },
    'Produto 3':{
        'Fresadora':5,
        'Torno':0,
        'Retificadora':2
    }
}

lucro_unitario = {
    'Produto 1':25,
    'Produto 2':25,
    'Produto 3':25
}

#objetivo
def lucro_total(model):
    return sum(model.x[p] * lucro_unitario[p] for p in model.produtos)
model.objetivo = pyo.Objective(rule=lucro_total, sense=pyo.maximize)

#Restrições
def restrição_tempo(model,m):


    return sum(model.x[p] * horas_necessarias[p][m] for p in model.produtos) <= tempo_disponivel[m]
model.restricao_tempo = pyo.Constraint(model.maquinas, rule=restrição_tempo)

def restricao_produto3(model):
    return model.x['Produto 3'] <= 20
model.restricao_produto3 = pyo.Constraint(rule=restricao_produto3)



In [59]:
model.pprint()

2 Set Declarations
    maquinas : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Fresadora', 'Torno', 'Retificadora'}
    produtos : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Produto 1', 'Produto 2', 'Produto 3'}

1 Var Declarations
    x : Size=3, Index=produtos
        Key       : Lower : Value : Upper : Fixed : Stale : Domain
        Produto 1 :     0 :  None :  None : False :  True : NonNegativeIntegers
        Produto 2 :     0 :  None :  None : False :  True : NonNegativeIntegers
        Produto 3 :     0 :  None :  None : False :  True : NonNegativeIntegers

1 Objective Declarations
    objetivo : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 25*x[Produto 1] + 25*x[Produto 2] + 25*x[Produto 3]

2 Constraint Declarations
    restricao_produto3 : Size=1, 

In [60]:
opt = SolverFactory('cplex', executable='C:\\CPLEX_Studio2211\\cplex\\bin\\x64_win64\\cplex.exe')
opt.options['TimeLimit'] = 180 
results = opt.solve(model, tee=True, warmstart=True)
print("-------FIM----------")



Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\joaon\AppData\Local\Temp\tmp4dgn0fif.cplex.log' open.
CPLEX> New value for time limit in seconds: 180
CPLEX> Problem 'C:\Users\joaon\AppData\Local\Temp\tmphznuxysu.pyomo.lp' read.
Read time = 0.00 sec. (0.00 ticks)
CPLEX> Warning: no MIP start values read, no MIP start loaded.
MIP start file 'C:\Users\joaon\AppData\Local\Temp\tmp7bjl_jjp.cplex.mst' read.
CPLEX> Problem name         : C:\Users\joaon\AppData\Local\Temp\tmphznuxysu.pyomo.lp
Objective sense      : Maximize
Variables            :       3  [General Integer: 3]
Objective nonzeros   :       3
Linear constraints   :       4  [

In [61]:
for p in model.produtos:
    print(f"{p}: {pyo.value(model.x[p])}")

print(f"Lucro Total: {pyo.value(model.objetivo)}")

Produto 1: 0.0
Produto 2: 87.0
Produto 3: 20.0
Lucro Total: 2675.0
